# DeepONet JAX Tutorial

The JAX tutorial of DeepONet to learn the antiderivative operator [here](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/neural_operators/simple_deepOnet_in_JAX.ipynb).